In [2]:
import chromadb
from chromadb import Settings

client = chromadb.HttpClient(settings=Settings(chroma_client_auth_provider="chromadb.config.BasicAuthClientProvider", chroma_client_auth_provider_config={"username": "admin", "password": "admin"}))
client.heartbeat()


provider:  chromadb.config.BasicAuthClientProvider


Exception: {"error":"Unauthorized"}